# Compile the pretrained PyTorch model with SageMaker Neo for On-device
---

이 노트북에서는 사전 훈련된 MnasNet 기반 이미지 분류(Image classification) 모델을 SageMaker Neo로 컴파일하여 배포합니다. SageMaker Neo는 머신 러닝 모델을 하드웨어에 맞게 최적화하는 API로, Neo로 컴파일한 모델은 클라우드와 엣지 디바이스 어디에서나 실행할 수 있습니다.

SageMaker Neo에서 지원하는 인스턴스 유형, 하드웨어 및 딥러닝 프레임워크는 아래 링크를 참조하세요.
(본 예제 코드는 2021년 2월 기준으로 작성되었으며, 작성 시점에서 PyTroch 1.8.0까지 지원하고 있습니다. 단, AWS Inferentia 기반 인스턴스로 배포 시에는 PyTorch 1.7.1까지 지원합니다.)

SageMaker Neo가 지원하는 인스턴스 타입, 하드웨어 및 딥러닝 프레임워크는 아래 링크를 참조해 주세요.
- 클라우드 인스턴스: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-cloud.html
- 엣지 디바이스: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-devices-edge.html

In [ ]:
%load_ext autoreload
%autoreload 2
%store -r
%store

In [ ]:
import logging, sys
def _get_logger():
    '''
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  

logger = _get_logger()

In [ ]:
import os, sys, sagemaker
sys.path.insert(0, "./src")
print(sagemaker.__version__)
model_trace_name = 'model.pth'
sample_img_path = "samples"

<br>

## 1. Inference script
---

아래 코드 셀은 SageMaker 추론 스크립트를 `src` 디렉토리에 저장합니다.

In [ ]:
%%writefile src/infer_pytorch_neo.py

import io
import json
import logging
import os
import pickle
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image  # Training container doesn't have this package

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


def model_fn(model_dir):
    import neopytorch

    logger.info("model_fn")
    neopytorch.config(model_dir=model_dir, neo_runtime=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # The compiled model is saved as "compiled.pt"
    model = torch.jit.load(os.path.join(model_dir, "compiled.pt"), map_location=device)

    # It is recommended to run warm-up inference during model load
    sample_input_path = os.path.join(model_dir, "sample_input.pkl")
    with open(sample_input_path, "rb") as input_file:
        model_input = pickle.load(input_file)
    if torch.is_tensor(model_input):
        model_input = model_input.to(device)
        model(model_input)
    elif isinstance(model_input, tuple):
        model_input = (inp.to(device) for inp in model_input if torch.is_tensor(inp))
        model(*model_input)
    else:
        print("Only supports a torch tensor or a tuple of torch tensors")

    return model
    
    
def transform_fn(model, payload, request_content_type='application/octet-stream', 
                 response_content_type='application/json'):

    logger.info('Invoking user-defined transform function')

    if request_content_type != 'application/octet-stream':
        raise RuntimeError(
            'Content type must be application/octet-stream. Provided: {0}'.format(request_content_type))

    # preprocess
    decoded = Image.open(io.BytesIO(payload))
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[
                0.485, 0.456, 0.406], std=[
                0.229, 0.224, 0.225]),
    ])
    normalized = preprocess(decoded)
    batchified = normalized.unsqueeze(0)

    # predict
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batchified = batchified.to(device)
    result = model.forward(batchified)

    # Softmax (assumes batch size 1)
    result = np.squeeze(result.detach().cpu().numpy())
    result_exp = np.exp(result - np.max(result))
    result = result_exp / np.sum(result_exp)

    response_body = json.dumps(result.tolist())

    return response_body, response_content_type

<br>

## 2. Load trained model
---

사전 훈련된 모델을 로드합니다. 다른 프레임워크 버전과의 호환성 문제 및 직렬화 중 문제를 줄이기 위해 가능한 한 전체 모델을 로드하는 것보다 모델 구조를 먼저 초기화하고 모델 가중치(weight)를 로드하는 것을 권장합니다.

In [ ]:
import torch
import torchvision.models as models
import tarfile
import src.infer_utils as infer_utils

classes_dict = infer_utils.load_classes_dict('classes_dict_imagenet.json')
num_classes = len(classes_dict)

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.mnasnet1_0(pretrained=True)
#model = models.mobilenet_v2(pretrained=True)
model = model.to(device)

In [ ]:
import torch
import torchvision.models as models
import tarfile

input_shape = [1,3,224,224]
dummy_input = torch.zeros(input_shape).float()
dummy_input = dummy_input.to(device)
trace = torch.jit.trace(model.float().eval(), dummy_input)
trace.save(model_trace_name)

### Local Inference without Endpoint

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있기 때문에, 로컬 환경 상에서 추론을 수행하면서 디버깅하는 것을 권장합니다. 아래 코드 셀의 코드를 예시로 참조해 주세요.

In [ ]:
import os
import json
import random
import numpy as np
from io import BytesIO
from PIL import Image
import src.infer_utils as infer_utils
from src.infer_pytorch_neo import transform_fn

model = torch.jit.load(model_trace_name)
model = model.to(device)

img_list = os.listdir(sample_img_path)
img_path_list = [os.path.join(sample_img_path, img) for img in img_list]

In [ ]:
test_idx = random.randint(0, len(img_list)-1)
img_path = img_path_list[test_idx]

with open(img_path, mode='rb') as file:
    payload = bytearray(file.read())
    
response_body, _ = transform_fn(model, payload)
result = json.loads(response_body)
infer_utils.parse_result(result, classes_dict, img_path, show_img=True)

<br>

## 3. Compile Model with SageMaker Neo
---

### Overview

Neo-AI는 다양한 머신 러닝 프레임워크를 지원하며 정확도 손실을 최소화하면서 자동으로 모델을 최적화합니다. Neo-AI 컴파일러는 타겟 디바이스의 OS 및 하드웨어 플랫폼에 맞게 모델을 자동으로 최적화하고 딥 러닝 런타임에서 모델을 실행 가능한 형태로 변환합니다. 딥러닝 런타임은 머신 러닝 프레임워크와 엣지 디바이스에 상관없이 단 두 줄의 코드로 추론을 수행할 수 있으며 런타임ㅁ 버전은 지속적으로 업데이트됩니다.

그리고 AWS 계정이 있다면 Neo-AI 기반의 관리형 서비스인 Amazon SageMaker Neo를 사용할 수 있습니다. SageMaker Neo는 간단한 API 호출이나 UI로 추가 패키지나 인프라 설정 및 요금 부과 없이 동시에 여러 타켓 디바이스들에 적합한 모델을 컴파일할 수 있습니다.

### (Deep Dive) SageMaker Neo Stack

SageMaker Neo Stack을 자세히 살펴보겠습니다. Neo는 먼저 컴파일러로 모델을 최적화한 다음, 클라우드 또는 온디바이스에서 런타임을 통해 컴파일된 모델을 실행합니다.

### Compiler
컴파일러 부분의 경우, 연산 그래프(computational graph) 제너레이터는 다양한 딥러닝 플랫폼에서 훈련된 딥러닝 모델을 로드하여 그래프 구조로 재구성합니다. 그런 다음, 모델에 정의된 연산자를 기본 연산자로 변환하여 계산 그래프를 생성합니다. 그래프 생성 후, 다양한 그래프 최적화 기법을 적용하여 최적화된 그래프를 생성합니다.

그러나, 모델이 작동할 타겟 하드웨어 아키텍처를 고려한 하드웨어 종속 최적화는 최적화된 그래프만으로는 불가능합니다. 따라서, 연산 그래프를 하드웨어 의존적 최적화의 한 형태인 IR(Intermediate Representation)로 변환할 필요가 있습니다. Relay IR은 생성된 IR을 기반으로 메모리 할당, 병렬화 및 실행 순서와 같은 하드웨어 종속 최적화를 수행한 후 코드를 생성합니다. 자세한 사항은 Relay IR 논문(https://arxiv.org/pdf/1810.00952.pdf)을 참조하세요.

Apache TVM이 항상 사용되는 것은 아닙니다. 딥러닝 프레임워크나 하드웨어 사양에 따라 TensorRT나 TreeLite를 사용합니다. NVIDIA GPU의 경우 Neo는 TensorRT를 사용합니다.

마지막으로, 백엔드 코드 생성기는 IR을 기반으로 딥러닝 모델 워크로드가 배포되는 대상 하드웨어 아키텍처(CPU, GPU, TPU 등)에 최적화된 백엔드 코드를 생성합니다.


### Runtime
런타임 부분은 DLR을 통해 수행됩니다.

![sm-neo-stack](../imgs/sm-neo-stack.png)


In [ ]:
with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add(model_trace_name)

### Model Compilation

아래 코드 셀은 배포 유형에 따른 다양한 유즈케이스를 고려하여, 아래의 5가지 유즈케이스에 대한 컴파일 job을 동시에 시작합니다.

- Cloud (CPU, `ml_m5` instance)
- Cloud (CPU, `ml_c5` instance)
- Cloud (GPU, `ml_g4dn` instance)
- NVIDIA Jetson nano (CPU)
- NVIDIA Jetson nano (GPU)

NVIDIA Jetpack에 따라 디바이스의 CUDA 버전 또는 TensorRT 버전이 호환되지 않을 수 있으며, GPU 모델을 로드하는 데 수십 초가 걸리므로 CPU 모델을 함께 컴파일하고 테스트하는 것이 좋은 전략입니다.

컴파일은 약 4-6분이 소요됩니다.

**[Caution] 컴파일 중 오류가 발생하면 이 코드를 실행하는 노트북의 PyTorch 버전을 반드시 확인히세요. PyTorch 버전이 일치해야 합니다. 이 실습에서는 PyTorch 1.8을 사용합니다.**

In [ ]:
import time, boto3, sagemaker
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

# For cloud ML inference
compilation_job_cloud_cpu_m5 = infer_utils.compile_model_for_cloud(
    role, bucket, target_device='ml_m5', dataset_dir=None, framework_version='1.8'
)
compilation_job_cloud_cpu_c5 = infer_utils.compile_model_for_cloud(
    role, bucket, target_device='ml_c5', dataset_dir=None, framework_version='1.8'
)
compilation_job_cloud_gpu = infer_utils.compile_model_for_cloud(
    role, bucket, target_device='ml_g4dn', dataset_dir=None, framework_version='1.8'
)

# For on-device ML inference
compilation_job_jetson_cpu = infer_utils.compile_model_for_jetson(
    role, bucket, dataset_dir=None, use_gpu=False
)
compilation_job_jetson_gpu = infer_utils.compile_model_for_jetson(
    role, bucket, dataset_dir=None, use_gpu=True
)

In [ ]:
compilation_jobs = [compilation_job_cloud_cpu_m5, compilation_job_cloud_cpu_c5, compilation_job_cloud_gpu, 
                    compilation_job_jetson_cpu, compilation_job_jetson_gpu]

In [ ]:
sm_client = boto3.client('sagemaker')

max_time = time.time() + 15*60 # 15 mins
for job in compilation_jobs:
    while time.time() < max_time:
        resp = sm_client.describe_compilation_job(CompilationJobName=job['job_name'])    
        if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
            print('Running...')
        else:
            print(resp['CompilationJobStatus'], job)
            break
        time.sleep(30)

### Review Compilation Jobs on AWS Console

In [ ]:
from IPython.core.display import display, HTML
region = boto3.Session().region_name

for job in compilation_jobs:
    job_name = job['job_name']
    display(
        HTML(
            '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/compilation-jobs/{}">Compilation Job</a> for {}</b>'.format(
                region, job_name, job_name
            )
        )
    )

## Copy Compiled model to local

아래 코드 셀은 컴파일된 모델을 S3에서 로컬로 복사합니다. 클라우드의 경우 인스턴스의 엔드포인트를 생성하여 실시간 배포가 가능하며, NVIDIA Jetson nano와 같은 온디바이스의 경우 모델을 디바이스에 복사하고 DLR을 설치합니다. DLR을 사용하면 PyTorch 및 TensorFlow와 같은 별도의 프레임워크를 설치할 필요 없이 간단한 API 호출로 모델을 쉽게 추론할 수 있습니다.

- Installing DLR: https://neo-ai-dlr.readthedocs.io/en/latest/install.html

In [ ]:
model_root_path = 'neo-model'
!rm -rf {model_root_path}
for job in compilation_jobs:
    model_path = f"{model_root_path}/{job['job_name']}"
    os.makedirs(model_path, exist_ok=True)
    !aws s3 cp {job['s3_compiled_model_path']} {model_path} --recursive 

<br>

## 4. On-device Deployment

컴파일된 모델을 AWS IoT Greengrass 컴포넌트로 배포하는 경우 아래 코드 셀의 예시처럼 출력값을 기록해 두세요. 온디바이스에서 테스트하려면 아래 쉘 명령어의 예시처럼 실행하시면 됩니다.

```shell
rm -rf model_cpu
mkdir model_cpu && cd model_cpu
aws s3 cp [MODEL-CLOUD-CPU-S3-PATH] . --recursive
tar -xzvf model-ml_m5.tar.gz && rm model-ml_m5.tar.gz
```

In [ ]:
model_cloud_cpu_s3_path = compilation_jobs[0]['s3_compiled_model_path']
print(model_cloud_cpu_s3_path)

모델을 온디바이스로 복사한 후 압축을 해제하였다면, 아래 스크립트를 온디바이스로 복사&수정하여 추론을 수행해 봅니다.

```python
import logging, sys
import cv2
import glob
import json
import numpy as np
import dlr
from dlr import DLRModel


def load_classes_dict(filename='classes_dict.json'):
    with open(filename, 'r') as fp:
        classes_dict = json.load(fp)

    classes_dict = {int(k):v for k,v in classes_dict.items()}        
    return classes_dict
    

def load_image(image_path):
    image_data = cv2.imread(image_path)
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    return image_data


def preprocess_image(image, image_shape=(224,224)):
    cvimage = cv2.resize(image, image_shape)
    img = np.asarray(cvimage, dtype='float32')
    img /= 255.0 # scale 0 to 1
    mean = np.array([0.485, 0.456, 0.406]) 
    std = np.array([0.229, 0.224, 0.225])
    img = (img - mean) / std
    img = np.transpose(img, (2,0,1)) 
    img = np.expand_dims(img, axis=0) # e.g., [1x3x224x224]
    return img


def softmax(x):
    x_exp = np.exp(x - np.max(x))
    f_x = x_exp / np.sum(x_exp)
    return f_x


device = 'cpu'
model = DLRModel(f'model_{device}', device)
sample_image_dir = 'sample_images'
classes_dict = load_classes_dict('classes_dict.json')

extensions = (f"{sample_image_dir}/*.jpg", f"{sample_image_dir}/*.jpeg")
img_filelist = [f for f_ in [glob.glob(e) for e in extensions] for f in f_]
print(img_filelist)

for img_filepath in img_filelist[:-1]:
    ground_truth = img_filepath.split('/')[-1]
    img = load_image(img_filepath)
    img_data = preprocess_image(img)
    
    output = model.run(img_data)  
    probs = softmax(output[0][0])
    sort_classes_by_probs = np.argsort(probs)[::-1]

    idx = sort_classes_by_probs[0]
    print("+"*80)
    print(f'predicted = {classes_dict[idx]}, {probs[idx]*100:.2f}%')
    print(f'ground_truth = {ground_truth}')  
```    